In [1]:
import os
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
# mlp
from sklearn.neural_network import MLPClassifier

In [11]:
labels_root = "./data/labels"

# Your local path to the NAIP embeddings data (or a subset/subdirectory of it)
# to run on all downloaded data, remove "wa" from the path
root = "/mnt/c/data/clay_naip/wa"

In [ ]:
# Load the labels
true_labels = []
false_labels = []
for lroot, dirs, files in os.walk(labels_root):
    for file in files:
        if "log" in file:
            true_labels.append(gpd.read_file(os.path.join(lroot, file)))
        else:
            false_labels.append(gpd.read_file(os.path.join(lroot, file)))

# concatenate all files
true_labels = gpd.GeoDataFrame(pd.concat(true_labels, ignore_index=True))
false_labels = gpd.GeoDataFrame(pd.concat(false_labels, ignore_index=True))

In [22]:
true_labels.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
# combine into single dataframe with label column of 0 and 1 (1 for log yard, 0 for null)
true_labels["label"] = 1
false_labels["label"] = 0

# combine into single dataframe
labels = pd.concat([true_labels, false_labels], ignore_index=True)

In [14]:
# Get training data from parquet files

def read_parquet_files(root, intersecting_gdf=None, sample_frac=None):
    
    files = []
    for root, dirs, filenames in os.walk(root):
        for filename in filenames:
            if filename.endswith(".parquet"):
                files.append(os.path.join(root, filename))

    geoms = []
    select_embeddings = []
    random_embeddings = []
    for i, file in enumerate(tqdm(files)):

        # read the parquet file
        df = gpd.read_parquet(file)
        # append the geometry and embedding to the lists
        geoms.extend(df.geometry.tolist())
        if intersecting_gdf is not None:
            # check for intersection with the provided gdf
            # use sjoin
            df.crs = "EPSG:4326"
            intersecting_rows = gpd.sjoin(
                df, intersecting_gdf, how="inner", predicate="intersects"
            )
            if len(intersecting_rows) > 0:
                select_embeddings.append(intersecting_rows)
            
        # sample
        if sample_frac is not None:
            # sample the rows
            sampled_rows = df.sample(frac=sample_frac)
            # append the sampled rows to the list
            random_embeddings.append(sampled_rows) 

    # return the 2 gdfs
    main_gdf = gpd.GeoDataFrame(geometry=geoms, crs="EPSG:4326")
    if len(select_embeddings) > 0:
        # concatenate the list of dataframes into a single dataframe
        select_embeddings = pd.concat(select_embeddings)

    if len(random_embeddings) > 0:
        # concatenate the list of dataframes into a single dataframe
        random_embeddings = pd.concat(random_embeddings)

    return main_gdf, select_embeddings, random_embeddings

# read the parquet files
main_gdf, logyard_embeddings_gdf, random_negative_embeddings = read_parquet_files(root, labels, sample_frac=0.001)

100%|██████████| 3184/3184 [04:52<00:00, 10.90it/s]


In [15]:
# combine the random negative embeddings
random_negative_embeddings["label"] = 0

# combine with sawmill embeddings
logyard_embeddings_gdf = pd.concat([logyard_embeddings_gdf, random_negative_embeddings], ignore_index=True)

In [21]:
logyard_embeddings_gdf.label.value_counts()

label
0    7340
1     972
Name: count, dtype: int64

In [23]:
# save to gdf
logyard_embeddings_gdf = gpd.GeoDataFrame(logyard_embeddings_gdf, geometry=logyard_embeddings_gdf.geometry)
logyard_embeddings_gdf.crs = "EPSG:4326"
logyard_embeddings_gdf.to_file("logyard_embeddings.gpkg", driver="GPKG")

In [16]:
# Prepare data for training
X = np.array([np.array(x) for x in logyard_embeddings_gdf.embeddings.tolist()])
y = logyard_embeddings_gdf.label.values
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
# create the model
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000, random_state=42)

# fit the model
model_rf.fit(X_train, y_train)
model_mlp.fit(X_train, y_train)
# make predictions
y_pred_rf = model_rf.predict(X_test)
y_pred_mlp = model_mlp.predict(X_test)
# print the classification report
print("RF:", classification_report(y_test, y_pred_rf))
print("MLP:", classification_report(y_test, y_pred_mlp))

RF:               precision    recall  f1-score   support

           0       0.89      0.90      0.89      1459
           1       0.23      0.22      0.23       204

    accuracy                           0.81      1663
   macro avg       0.56      0.56      0.56      1663
weighted avg       0.81      0.81      0.81      1663

MLP:               precision    recall  f1-score   support

           0       0.89      0.90      0.90      1459
           1       0.24      0.23      0.23       204

    accuracy                           0.82      1663
   macro avg       0.56      0.56      0.56      1663
weighted avg       0.81      0.82      0.81      1663



In [18]:
# apply model per parquet file and collate results
def apply_model_to_parquet(file, model_rf, model_mlp, scaler):
    # read the parquet file
    df = gpd.read_parquet(file)
    # get the embeddings
    X = np.array([np.array(x) for x in df.embeddings.tolist()])
    # scale the data
    X = scaler.transform(X)
    # make predictions
    y_pred_rf = model_rf.predict(X)
    y_pred_mlp = model_mlp.predict(X)
    # add the predictions to the dataframe
    df["predictions_rf"] = y_pred_rf
    df["predictions_mlp"] = y_pred_mlp

    # drop the embeddings column
    df = df.drop(columns=["embeddings"])

    return df

# List all files in the data directory
files = []
for root, dirs, filenames in os.walk(root):
    for filename in filenames:
        if filename.endswith(".parquet"):
            files.append(os.path.join(root, filename))

results = []
for file in tqdm(files):
    # read the parquet file
    df = gpd.read_parquet(file)
    # apply the model
    df = apply_model_to_parquet(file, model_rf, model_mlp, scaler)
    # append to results
    results.append(df)

 55%|█████▍    | 1744/3184 [07:13<05:57,  4.03it/s]


KeyboardInterrupt: 

In [ ]:
# concatenate all results into a single dataframe
results = pd.concat(results, ignore_index=True)
# save the results
results_file = "model_predictions.gpkg"
results.to_file(results_file, driver="GPKG")